In [ ]:
# %pip install lm-format-enforcer -q

In [18]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained('./llm')
llm = LLM(
    model='./llm',
    dtype=torch.bfloat16,
    gpu_memory_utilization=0.9,
    max_seq_len_to_capture=8192,
    max_model_len=8192,
    # enforce_eager=True
)

WARNING 09-08 00:57:58 config.py:1651] Casting torch.float16 to torch.bfloat16.
INFO 09-08 00:57:58 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='./llm', speculative_config=None, tokenizer='./llm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./llm, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/14 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   7% Completed | 1/14 [01:44<22:32, 104.05s/it]
Loading safetensors checkpoint shards:  14% Completed | 2/14 [03:30<21:04, 105.40s/it]
Loading safetensors checkpoint shards:  21% Completed | 3/14 [03:47<11:57, 65.26s/it]
Loading safetensors checkpoint shards:  29% Completed | 4/14 [05:33<13:32, 81.24s/it]
Loading safetensors checkpoint shards:  36% Completed | 5/14 [07:20<13:33, 90.41s/it]
Loading safetensors checkpoint shards:  43% Completed | 6/14 [09:06<12:47, 95.94s/it]
Loading safetensors checkpoint shards:  50% Completed | 7/14 [10:51<11:31, 98.76s/it]
Loading safetensors checkpoint shards:  57% Completed | 8/14 [12:38<10:07, 101.32s/it]
Loading safetensors checkpoint shards:  64% Completed | 9/14 [14:29<08:41, 104.33s/it]
Loading safetensors checkpoint shards:  71% Completed | 10/14 [16:15<06:59, 105.00s/it]
Loading safetensors checkpoint shards:  79% Completed | 

INFO 09-08 01:24:34 model_runner.py:926] Loading model weights took 60.1881 GB
INFO 09-08 01:25:58 gpu_executor.py:122] # GPU blocks: 3159, # CPU blocks: 1638
INFO 09-08 01:25:59 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-08 01:25:59 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-08 01:26:26 model_runner.py:1335] Graph capturing finished in 27 secs.


In [95]:
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.vllm import (
    build_vllm_logits_processor,
    build_vllm_token_enforcer_tokenizer_data
)
from typing import Union, List, Optional, Dict
from vllm import SamplingParams

ListOrStrList = Union[str, List[str]]

tokenizer_data = build_vllm_token_enforcer_tokenizer_data(llm)

genetation_kwargs = {
    'temperature': 0.2,
    'top_p': 0.7,
    # 'top_k': 30,
    'max_tokens': 1024,
    'repetition_penalty': 1.12
}
def vllm_with_character_level_parser(
    user_prompt: ListOrStrList,
    genetation_kwargs: Optional[Dict[str, Union[int, float]]] = None,
    parser: Optional[CharacterLevelParser] = None
) -> ListOrStrList:
    
    sampling_params = SamplingParams() if genetation_kwargs is None else SamplingParams(**genetation_kwargs)

    if parser:
        logits_processor = build_vllm_logits_processor(tokenizer_data, parser)
        sampling_params.logits_processors = [logits_processor]
    # results = llm.generate(prompt, sampling_params=sampling_params)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]
    
    prompt = llm.llm_engine.tokenizer.tokenizer.apply_chat_template(
        conversation=messages,
        add_generation_prompt=True,
        tokenize=False
    )
    prompts = [prompt]
    
    results = llm.generate(prompts, sampling_params)
    
    if isinstance(user_prompt, str):
        return results[0].outputs[0].text
    else:
        return [result.outputs[0].text for result in results]

In [96]:
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
from typing import List, Tuple
import datetime

"""
Структура JSON файла, как будут генерироваться ответы LLM
Сделано для упрощения пост процессинга данных
"""
class AnswerFormat(BaseModel):
    conference_name: str # название совещания
    conference_date: datetime.date # дата проведения совещания
    users_name: List[str] # имена участников конференции
    user_status: List[str] # должности участников конференции
    question_title: List[str] # название вопроса
    questions: List[str] # вопросы, обсуждаемые на совещании
    solutions: List[str] # решения по каждому отдельному вопросу
    responsible_person: List[str] # список лиц, ответственных за задания
    time_for_solutions: List[datetime.date] # крайний срок решения задачи
    

In [106]:
USER_PROMPT = """
Ты занимаешься поиском важной информации из расшифровок совещаний. Тебе необходимо найти во входном тексте следующую информацию:
1. Название совещания
2. Дата совещания
3. ФИО всех участников совещания
4. Должности всех участников совещания
5. Краткое наименование рассмотренных задач
6. Какие задачи были рассмотрены на совещании
7. Какое решение предполагается по каждому вопросу
8. Список лиц, ответственных за выполнение задачи
9. Крайний срок решения задачи

При поиске информации соблюдай следующие правила:
1. Информации может не быть в тексте. Если такое случилось, НЕ ПРИДУМЫВАЙ ТЕКСТ ОТ СЕБЯ, а напиши, что информации в тексте не найдено
2. Ответ в тексте может быть представлен в неполном виде (например, вместо ФИО участника совещания указано только его имя, а должность может быть не указана). В таком случае напиши ту информацию, что известна достоверно
3. В случае, если у человека не указана его должность, в ответ напиши "Должность_неизвестна"
4. При ответе на пункт 2 (название совещания) напиши краткий смысл того, для чего данное совещание устраивалось. При ответе на данный вопрос используй не более 5 слов
Например: Рассмотрение апелляции, Кадровая политика, Стратегическая сессия, Итоги судебного заседания 
5. При ответе на пункт 5 (Краткое наименование рассмотренных задач) опиши кратко смысл каждоый из рассмотренных задач
6. При ответе на пункт 6 (Какие задачи были рассмотрены на совещании) опиши ПОДРОБНО, какие задачи были приняты к рассмотрению
7. Ответы на пункты 7, 8 и 9 (Какое решение предполагается по каждому вопросу, Список лиц, ответственных за выполнение задачи, Крайний срок решения задачи) могут не присутствовать в тексте. Если ответов на данные вопросы нет, напиши "неизвестно" в каждое поле
8. Количество строк в пунктах 6, 7, 8 и 9 должно совпадать
9. При ответе на пункты 3, 4 (ФИО всех участников совещания, Должности всех участников совещания) НЕ ДОБАВЛЯЙ людей, которые НЕ УЧАСТВУЮТ в совещании напрямую.
10. Ты ДОЛЖЕН вывести информацию в следующей json схеме: {json_schema}

Ниже тебе даны примеры, какой текст поступает на вход и какой ответ должен быть правильным:
=================================
Пример текста: Апелляционный суд посредством видеоконференцсвязи рассматривает апелляционную жалобу главы Крестьянскофермерское хозяйство Дубилина александра Геннадьевича, поданную на решение Арбитражного суда Волгоградской области 12 декабря 2012го года по делу а1220042602012 по иску главы КФХ Дубилина александра геннадьевича к администрации муниципального района Волгоградской области с участием третьего лица – Стародымого сергея ивановича о взыскании денежных средств в виде убытков. Пожалуйста, уважаемые коллеги представьтесь, кто осуществляет помощью в проведении и видеоконференцсвязь со стороны Арбитражного суда Волгоградской области. Судебное поручение исполняет судья Сотникова. Протокол судебного заседания ведет помощник судьи – Наумкина. У нас явилась представитель истца. Ответчик и третье лицо извещены надлежащим образом. Явка представителям не обеспечили. Стороны у нас извещены надлежащим образом.
Правильный вывод:
1. Название совещания: рассмотрение апелляционной жалобы
2. Дата совещания: неизвестна
3. ФИО всех участников совещания: Дубилин Александр Геннадиевич, Стародымов Сергей Иванович,  Сотникова, Наумкина
4. Должности всех участников совещания: заявитель, обвиняемый, судья, помощник судьи
5. Краткое наименование рассмотренных задач: апелляционная жалоба
6. Какие задачи были рассмотрены на совещании: была рассмотрена жалоба о взыскании денежных средств в виде убытков
7. Какое решение предполагается по каждому вопросу: неизвестно
8. Список лиц, ответственных за выполнение задачи: неизвестно
9. Крайний срок решения задачи: неизвестно


Текст для поиска информации:
Давайте задокументируем дату собрания - 27 сентября 2023 года. Да, здравствуйте. Яна, Аня и Егор. Сегодня мы на троих проводим совещание. Расскажите, пожалуйста, про ваше ощущение от дня, от предыдущего совещания, которое мы записали. Сделаем выводы, поставим уроки. Ну и что вам завтра хорошенького хотелось бы сделать для нашей организации. Ну и для нашей команды. Сегодня вы познакомились со многими. Расскажите ваши ощущения. Аня, начните. По алфавиту вы первая. Так несправедливо. Справедливо. Ну, за сегодняшний день мы закончили практически кейс на фотон. Но осталась довольно, конечно, большая часть. Но я думаю, мы управимся до завтрашнего дня и сделаем большое дело для Омской области. На завтра, наверное, как раз таки хотелось его закончить и закончить речь. Я на вас... На завтра вы не завершите речи. Хакатон — это очень обширная история. Там надо посидеть, там ещё денёк обменить, чтобы она сложилась. Ну и вообще, конечно, это будет качественное решение. То есть если мы реально предложим датасет, который состоит из трёх частей аудиозаписи, потом расшифровка и протокол встречи, будет вообще толстый. Будет прям пускобомба. Нам действительно нужно 15 лет, мне кажется, это вообще на несколько дней. Ну, надо вот такие маленькие, типа 10 минут записывать какие-то, или сразу расшифровку брать и расшифровку эту потом спокойно прогонять через... Прям плохая расшифровка. Да, хорошо видно. То есть надо что-то править. То есть там пунктуации, конечно, нет, но надо что-то делать. Вот так. Так что это, ну, большая задача, которую надо закрыть в ближайшие дни. И потом сделать презу на речи. Я думаю, это две крупнейшие задачи на неделе. Презентацию сделать? Ну, по речи. Я скинул шаблон? Что? Ну, през это сокращение. Не поймешь. Да, презентация. През и презентация. Ну, мы записываем уже, а дальше разб погодные условия, шторм, ураган, сегодня был хороший день, мы много работали, познакомились с очень важными и интересными людьми, набрались опыта от общения, от коммуникации, от возможностей Зарядились мотивацией Много работали Какой у вас опыт от людей? Мы узнали о том Какие есть перспективы развития Какие? Мы начали бояться своего начальника Почему? Меньше выпендриваться и вы не начали. То есть вот так вот, мое самовыражение... Задача — наладить больше коннекта. Существует все-таки такое ограничение сотрудников, как бы не чувствуется возможность какого-то там личностного... Чего? Выпендрежа. Выпендрежа? Не хватает пространства для выпендрежа? Неправда, простите меня, я не права, я так больше никогда не буду, мне очень стыдно. До сих пор выпендриваете, вы понимаете, насколько у вас много пространства? Я так больше никогда не буду. Окей, не делайте. Ну и? В общем, сегодня мы работали над очень интересной задачей Мы продолжали разработку нашего кейса на Хакатон Это очень такая тяжелая работа, потому что приходилось анализировать большой поток информации Сложно осуществлять поиск, потому что информация все-таки Вы-то знаете, это глобальная с сеть, которой очень много разной информации, очень тяжело её фильтровать, чтобы она была полезна. Мы с этой сложной задачей почти справились. Далее мы занимались работой с новыми информационными цифровыми технологиями, узнали новый сайт, который прорабатывает видео в текст. Это было полезно. Далее мы занимались наиважнейшей работой мы слушали текст и сопоставляли его с чем но стран скреби раваны информацией вот потрясающий сегодняшний сегодня день считаю что он очень полезен был на завтра еще много задач конечно не. Не стоит забывать про то, что в первой половине дня мы выполнили очень важную срочную задачу. Мы составили экселевскую таблицу по перспективам в ФАИВах в сфере искус Это очень крутая тема. Я вам обратную связь, можно там уже? Мы обсудили после обеда, видели видео с Александром и Фёдором. Созванивались, чётко приняли, сказали, всё очень круто, и прямо по твоей таблице сверху до низу начали уже прям писать всем руководителям, типа, давайте с вами свяжемся, проработаем точно тонкости, как внедрить искусственный интеллект у вас, обратную связь, чтобы составить потом список и передать куда необходимо на ту сторону речки. Вот. А вторая часть, мы по опроснику, на него будет опираться руководитель при ведении беседы. Вот так. То есть у нас будет по факту отличный кейс для правительства, который мы сможем дальше прорабатывать. Благодаря вашему труду. Спасибо вам большое. Аня и Яна. Это было немногоства, который мы сможем дальше прорабатывать. Благодаря вашему труду, спасибо вам большое. Аня и Яна, это было немного времени, но мы классно организовали труд утром, правда, и там выделили сразу у Яны сильная сторона, это генерация информации, у тебя порядок, и порядок и технический склад ума, который, ну, это все как с формул все сделает, поэтому я подумал заранее, что будет правильно именно так разделить работу, и мы эффективнее справимся. А, мы с вами такая комбо. Да. А потом, можно я вам тонкость расскажу? Мне так, ну ладно, потом в конце практики расскажу эту всю новость. Не под записи. Окей. Не под записи. Да. Может, мы хотели переслушивать это? Я вечером буду слушать. Ну да, вот потом отдадут же разработчикам. Ну, короче, это очень крутая тема. Мы проработали важный момент, который реально полезен для Родины. Это первый план. Служим России. Служим России. Да. Коллеги, завтра у нас очень важный день. Что завтра делать будем? Нам предстоит закончить работу над Хакатон. Ну, продолжить работу над Хакатон. Да, закончить Хакатон, проанализировать еще очень много информации, подготовить набор информации, датасет. Мне кажется, мы завтра весь день этому посвятим. Надо нормально как-то нарезать. А, завтра еще встреча. За полдня на это это всё идёт, две встречи у меня. Завтра. Надо нарезать всю эту историю. — Ну, я сегодня вечером ещё допишу. — Да, сегодня... Да, я нашёл ссылку на совещание правительства по 15 минут. Там, типа, тоже есть высказывания, несколько руководителей. — Сейчас Лукашенко закончим. — Да, Лукашенко делайте, и потом Мишустин, там, остальные. — Да, Лукаш остальные, они выступают.
""".format(json_schema=AnswerFormat.schema_json())

In [107]:
result = vllm_with_character_level_parser(USER_PROMPT, genetation_kwargs, JsonSchemaParser(AnswerFormat.schema()))


Processed prompts: 100%|██████████| 1/1 [00:32<00:00, 32.54s/it, est. speed input: 100.09 toks/s, output: 17.55 toks/s]


In [108]:
from IPython.display import display, Markdown

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))
    
    
display_content(result)

```
{
    "conference_name": "Стратегическое планирование",
    "conference_date": "2023-09-27",
    "users_name": [
        "Яна", 
        "Аня", 
        "Егор"
    ],
    "user_status": [
        "участник", 
        "участник", 
        "участник"
    ],
    "question_title": [
        "Обзор прошлого совещания", 
        "Задачи на будущее", 
        "Команда и организация", 
        "Знакомство с коллегами", 
        "Перспективы развития", 
        "Опыт взаимодействия", 
        "Сложности в работе", 
        "Развитие навыков", 
        "Важные задачи", 
        "План на завтра"
    ],
    "questions": [
        "Какое впечатление осталось от предыдущего совещания и что можно улучшить в будущем", 
        "Что хотите сделать для организации и команды в ближайшее время", 
        "Расскажите о своих впечатлениях от сегодняшнего знакомства с коллегами", 
        "Какие перспективы развития видит команда", 
        "Какой опыт получили от взаимодействия с коллегами", 
        "Какие сложности возникли в процессе работы", 
        "Как развивать свои навыки и налаживать связи внутри команды", 
        "Какие важные задачи были выполнены сегодня", 
        "Какой план действий на завтра", 
        "Какие встречи запланированы на следующий день"
    ],
    "solutions": [
        "Анализировать прошлые совещания и делать выводы для улучшения", 
        "Определить цели и задачи для организации и команды", 
        "Продолжать знакомиться и взаимодействовать с коллегами", 
        "Обсуждать перспективы развития и возможности роста", 
        "Обмениваться опытом и знаниями", 
        "Решать возникающие проблемы совместно", 
        "Налаживать коммуникацию и создавать пространство для самореализации", 
        "Выполнить срочные задачи и подготовить презентацию", 
        "Завершить работу над текущими проектами", 
        "Запланировать встречи и распределить обязанности"
    ],
    "responsible_person": [
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники", 
        "Все участники"
    ],
    "time_for_solutions": [
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно", 
        "неизвестно"
    ]
}
```

In [ ]:
import torch

a = torch.rand(20, 20)
b = torch.rand(20, 200)

while 1:
    c = a @ b